In [1]:
import os
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms as T
import cv2
import pandas as pd

from self_sup_data.chest_xray import SelfSupChestXRay
from model.resnet import resnet18_enc_dec
from train_chest_xray import SETTINGS
from experiments.chest_xray_tasks import test_real_anomalies


def test(test_dat, setting, device, preact=False, pool=True, final=True, show=True, plots=False):
    if final:
        fname = os.path.join(model_dir, setting.get('out_dir'), 'final_' + setting.get('fname'))
    else: 
        fname = os.path.join(model_dir, setting.get('out_dir'), setting.get('fname'))
    print(fname)
    if not os.path.exists(fname):
        return np.nan, np.nan

    model = resnet18_enc_dec(num_classes=1, preact=preact, pool=pool, in_channels=1,
                             final_activation=setting.get('final_activation')).to(device)
    if final:
        model.load_state_dict(torch.load(fname))
    else:
        model.load_state_dict(torch.load(fname).get('model_state_dict'))
    
    if plots:
        sample_ap, sample_auroc, fig = test_real_anomalies(model, test_dat, 
                        device=device, batch_size=16, show=show, full_size=True)
        fig.savefig(os.path.join(out_dir, setting.get('fname')[:-3] + '.png'))
        plt.close(fig)
    else:
        sample_ap, sample_auroc, _ = test_real_anomalies(model, test_dat, 
                        device=device, batch_size=16, show=show, plots=plots, full_size=True)
    
    return sample_ap, sample_auroc


model_dir = 'put/your/path/here'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
out_dir = 'put/your/path/here'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

data_dir = 'put/your/path/here'

device = torch.device("cuda"  if torch.cuda.is_available() else "cpu")
print(f'Using {device}')

Using cuda


# Evaluation for male PA

In [ ]:
with open('self_sup_data/chest_xray_lists/norm_MaleAdultPA_test_curated_list.txt', "r") as f:
    normal_test_files = f.read().splitlines()
with open('self_sup_data/chest_xray_lists/anomaly_MaleAdultPA_test_curated_list.txt', "r") as f:
    anom_test_files = f.read().splitlines()

modes = list(SETTINGS.keys())

sample_ap_df = pd.DataFrame(columns=modes)
sample_auroc_df = pd.DataFrame(columns=modes)

test_dat = SelfSupChestXRay(data_dir=data_dir, normal_files=normal_test_files, anom_files=anom_test_files,
                             is_train=False, res=256, transform=T.CenterCrop(224))
sample_aps, sample_aurocs = {}, {}
for mode in modes:
    sample_aps[mode], sample_aurocs[mode] = test(
            test_dat, SETTINGS.get(mode), device, preact=False, pool=True, final=True)

sample_ap_df = sample_ap_df.append(sample_aps, ignore_index=True).transpose()
sample_auroc_df = sample_auroc_df.append(sample_aurocs, ignore_index=True).transpose()


In [6]:
pd.options.display.float_format = '{:3.1f}'.format

sample_table = 100 * sample_auroc_df.transpose()

cols_shift = sample_table.loc[: , 'Shift-M':'Shift-M-123425']
cols_shift_int = sample_table.loc[: , 'Shift-Intensity-M':'Shift-Intensity-M-123425']
cols_shift_raw_int = sample_table.loc[: , 'Shift-Raw-Intensity-M':'Shift-Raw-Intensity-M-123425']
cols_pii = sample_table.loc[: , 'FPI-Poisson':'FPI-Poisson-123425']

merge_func = lambda x: r'{:3.1f} {{\tiny $\pm$ {:3.1f}}}'.format(*x)
sample_table['Shift'] = list(map(merge_func, zip(cols_shift.mean(axis=1),  cols_shift.std(axis=1))))
sample_table['Shift-Intensity'] = list(map(merge_func, zip(cols_shift_int.mean(axis=1),  cols_shift_int.std(axis=1))))
sample_table['Shift-Raw-Intensity'] = list(map(merge_func, zip(cols_shift_raw_int.mean(axis=1),  cols_shift_raw_int.std(axis=1))))
sample_table['FPI-Poisson'] = list(map(merge_func, zip(cols_pii.mean(axis=1),  cols_pii.std(axis=1))))


sample_table = sample_table[['CutPaste', 'FPI', 'FPI-Poisson', 
                             'Shift', 'Shift-Raw-Intensity', 'Shift-Intensity']].rename(
    columns={'Shift':'Ours (binary)', 'Shift-Intensity':'Ours (logistic)', 'Shift-Raw-Intensity':'Ours (continous)'})

print(sample_table.to_latex(escape=False))

\begin{tabular}{lrrllll}
\toprule
{} &  CutPaste &  FPI &             FPI-Poisson &           Ours (binary) &        Ours (continous) &         Ours (logistic) \\
\midrule
0 &      59.8 & 73.7 &  91.7 {\tiny $\pm$ 0.6} &  94.0 {\tiny $\pm$ 0.5} &  93.4 {\tiny $\pm$ 0.3} &  94.0 {\tiny $\pm$ 0.6} \\
\bottomrule
\end{tabular}



In [7]:
sample_table  # male

,CutPaste,FPI,FPI-Poisson,Ours (binary),Ours (continous),Ours (logistic)
0,59.8,73.7,91.7 {\tiny $\pm$ 0.6},94.0 {\tiny $\pm$ 0.5},93.4 {\tiny $\pm$ 0.3},94.0 {\tiny $\pm$ 0.6}


# Evaluation for female PA

In [ ]:
with open('self_sup_data/chest_xray_lists/norm_FemaleAdultPA_test_curated_list.txt', "r") as f:
    normal_test_files = f.read().splitlines()
with open('self_sup_data/chest_xray_lists/anomaly_FemaleAdultPA_test_curated_list.txt', "r") as f:
    anom_test_files = f.read().splitlines()

modes = list(SETTINGS.keys())

sample_ap_df = pd.DataFrame(columns=modes)
sample_auroc_df = pd.DataFrame(columns=modes)

test_dat = SelfSupChestXRay(data_dir=data_dir, normal_files=normal_test_files, anom_files=anom_test_files,
                             is_train=False, res=256, transform=T.CenterCrop(224))
sample_aps, sample_aurocs = {}, {}
for mode in modes:
    sample_aps[mode], sample_aurocs[mode] = test(
            test_dat, SETTINGS.get(mode), device, preact=False, pool=True, final=True)

sample_ap_df = sample_ap_df.append(sample_aps, ignore_index=True).transpose()
sample_auroc_df = sample_auroc_df.append(sample_aurocs, ignore_index=True).transpose()

In [16]:
pd.options.display.float_format = '{:3.1f}'.format

sample_table = 100 * sample_auroc_df.transpose()

cols_shift = sample_table.loc[: , 'Shift-M':'Shift-M-123425']
cols_shift_int = sample_table.loc[: , 'Shift-Intensity-M':'Shift-Intensity-M-123425']
cols_shift_raw_int = sample_table.loc[: , 'Shift-Raw-Intensity-M':'Shift-Raw-Intensity-M-123425']
cols_pii = sample_table.loc[: , 'FPI-Poisson':'FPI-Poisson-123425']

merge_func = lambda x: r'{:3.1f} {{\tiny $\pm$ {:3.1f}}}'.format(*x)
sample_table['Shift'] = list(map(merge_func, zip(cols_shift.mean(axis=1),  cols_shift.std(axis=1))))
sample_table['Shift-Intensity'] = list(map(merge_func, zip(cols_shift_int.mean(axis=1),  cols_shift_int.std(axis=1))))
sample_table['Shift-Raw-Intensity'] = list(map(merge_func, zip(cols_shift_raw_int.mean(axis=1),  cols_shift_raw_int.std(axis=1))))
sample_table['FPI-Poisson'] = list(map(merge_func, zip(cols_pii.mean(axis=1),  cols_pii.std(axis=1))))


sample_table = sample_table[['CutPaste', 'FPI', 'FPI-Poisson', 
                             'Shift', 'Shift-Raw-Intensity', 'Shift-Intensity']].rename(
    columns={'Shift':'Ours (binary)', 'Shift-Intensity':'Ours (logistic)', 'Shift-Raw-Intensity':'Ours (continous)'})

print(sample_table.to_latex(escape=False))

\begin{tabular}{lrrllll}
\toprule
{} &  CutPaste &  FPI &             FPI-Poisson &           Ours (binary) &        Ours (continous) &         Ours (logistic) \\
\midrule
0 &      56.2 & 67.4 &  92.8 {\tiny $\pm$ 0.4} &  94.3 {\tiny $\pm$ 0.6} &  93.0 {\tiny $\pm$ 0.4} &  94.0 {\tiny $\pm$ 0.5} \\
\bottomrule
\end{tabular}



In [17]:
sample_table  # female

,CutPaste,FPI,FPI-Poisson,Ours (binary),Ours (continous),Ours (logistic)
0,56.2,67.4,92.8 {\tiny $\pm$ 0.4},94.3 {\tiny $\pm$ 0.6},93.0 {\tiny $\pm$ 0.4},94.0 {\tiny $\pm$ 0.5}
